### Importing Libraries

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
import numpy as np

### Reading csv file

In [2]:
data = pd.read_csv('heart_failure.csv')
data.head()

,Unnamed: 0,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT,death_event
0,0,75.0,no,582,no,20,yes,265000.00,1.9,130,yes,no,4,1,yes
1,1,55.0,no,7861,no,38,no,263358.03,1.1,136,yes,no,6,1,yes
2,2,65.0,no,146,no,20,no,162000.00,1.3,129,yes,yes,7,1,yes
3,3,50.0,yes,111,no,20,no,210000.00,1.9,137,yes,no,7,1,yes
4,4,65.0,yes,160,yes,20,no,327000.00,2.7,116,no,no,8,1,yes


### Inspecting Data

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                299 non-null    int64  
 1   age                       299 non-null    float64
 2   anaemia                   299 non-null    object 
 3   creatinine_phosphokinase  299 non-null    int64  
 4   diabetes                  299 non-null    object 
 5   ejection_fraction         299 non-null    int64  
 6   high_blood_pressure       299 non-null    object 
 7   platelets                 299 non-null    float64
 8   serum_creatinine          299 non-null    float64
 9   serum_sodium              299 non-null    int64  
 10  sex                       299 non-null    object 
 11  smoking                   299 non-null    object 
 12  time                      299 non-null    int64  
 13  DEATH_EVENT               299 non-null    int64  
 14  death_even

In [4]:
data.describe()

,Unnamed: 0,age,creatinine_phosphokinase,ejection_fraction,platelets,serum_creatinine,serum_sodium,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000
mean,149.000000,60.833893,581.839465,38.083612,263358.029264,1.39388,136.625418,130.260870,0.32107
std,86.458082,11.894809,970.287881,11.834841,97804.236869,1.03451,4.412477,77.614208,0.46767
min,0.000000,40.000000,23.000000,14.000000,25100.000000,0.50000,113.000000,4.000000,0.00000
25%,74.500000,51.000000,116.500000,30.000000,212500.000000,0.90000,134.000000,73.000000,0.00000
50%,149.000000,60.000000,250.000000,38.000000,262000.000000,1.10000,137.000000,115.000000,0.00000
75%,223.500000,70.000000,582.000000,45.000000,303500.000000,1.40000,140.000000,203.000000,1.00000
max,298.000000,95.000000,7861.000000,80.000000,850000.000000,9.40000,148.000000,285.000000,1.00000


### Data Preprocessing

In [5]:
Counter(data.death_event)

Counter({'yes': 96, 'no': 203})

### Labels and Features selection

In [6]:
y = data.death_event
x = data[['age','anaemia','creatinine_phosphokinase','diabetes','ejection_fraction','high_blood_pressure','platelets','serum_creatinine','serum_sodium','sex','smoking','time']]

In [7]:
x = pd.get_dummies(x)

### Data split into training and testing sets

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

### Standardize Numerical features

In [9]:
numerical_features = x.select_dtypes(include = ['float64', 'int64'])
numerical_columns = numerical_features.columns

In [10]:
ct = ColumnTransformer([('standardize', StandardScaler(), numerical_columns)], remainder = 'passthrough')

In [11]:
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

### Prepare labels for classification

In [12]:
le = LabelEncoder()

In [13]:
Y_train = le.fit_transform(Y_train.astype(str))
Y_test = le.transform(Y_test.astype(str))

In [14]:
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

### Design the model

In [15]:
model = Sequential()

In [16]:
model.add(InputLayer(input_shape = (X_train.shape[1], )))
model.add(Dense(12, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))

In [17]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

### Train and evaluate the model

In [18]:
model.fit(X_train, Y_train, epochs = 100, batch_size = 16, verbose = 1)

Epoch 1/100
14/14 [==============================] - 2s 6ms/step - loss: 1.2103 - accuracy: 0.2297
Epoch 2/100
14/14 [==============================] - 0s 7ms/step - loss: 1.1017 - accuracy: 0.2440
Epoch 3/100
14/14 [==============================] - 0s 7ms/step - loss: 1.0133 - accuracy: 0.2488
Epoch 4/100
14/14 [==============================] - 0s 7ms/step - loss: 0.9314 - accuracy: 0.2727
Epoch 5/100
14/14 [==============================] - 0s 6ms/step - loss: 0.8657 - accuracy: 0.3062
Epoch 6/100
14/14 [==============================] - 0s 7ms/step - loss: 0.8167 - accuracy: 0.3684
Epoch 7/100
14/14 [==============================] - 0s 6ms/step - loss: 0.7733 - accuracy: 0.4019
Epoch 8/100
14/14 [==============================] - 0s 7ms/step - loss: 0.7317 - accuracy: 0.4737
Epoch 9/100
14/14 [==============================] - 0s 7ms/step - loss: 0.6994 - accuracy: 0.5502
Epoch 10/100
14/14 [==============================] - 0s 7ms/step - loss: 0.6724 - accuracy: 0.6172
Epoch 11/

14/14 [==============================] - 0s 6ms/step - loss: 0.3311 - accuracy: 0.8565
Epoch 83/100
14/14 [==============================] - 0s 8ms/step - loss: 0.3308 - accuracy: 0.8517
Epoch 84/100
14/14 [==============================] - 0s 9ms/step - loss: 0.3297 - accuracy: 0.8565
Epoch 85/100
14/14 [==============================] - 0s 9ms/step - loss: 0.3295 - accuracy: 0.8565
Epoch 86/100
14/14 [==============================] - 0s 7ms/step - loss: 0.3291 - accuracy: 0.8565
Epoch 87/100
14/14 [==============================] - 0s 7ms/step - loss: 0.3291 - accuracy: 0.8612
Epoch 88/100
14/14 [==============================] - 0s 8ms/step - loss: 0.3282 - accuracy: 0.8565
Epoch 89/100
14/14 [==============================] - 0s 7ms/step - loss: 0.3270 - accuracy: 0.8565
Epoch 90/100
14/14 [==============================] - 0s 7ms/step - loss: 0.3258 - accuracy: 0.8565
Epoch 91/100
14/14 [==============================] - 0s 7ms/step - loss: 0.3251 - accuracy: 0.8612
Epoch 92/100


In [19]:
loss, acc = model.evaluate(X_test, Y_test, verbose = 0)

In [20]:
loss

0.42054468393325806

In [21]:
acc

0.8222222328186035

### Generating a classification report

In [22]:
y_estimate = model.predict(X_test, verbose = 0)
y_estimate = np.argmax(y_estimate, axis = 1)
y_estimate

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0], dtype=int64)

In [23]:
y_true = np.argmax(Y_test, axis = 1)
y_true

array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0], dtype=int64)

In [27]:
print(classification_report(y_true, y_estimate))

              precision    recall  f1-score   support

           0       0.83      0.94      0.88        62
           1       0.80      0.57      0.67        28

    accuracy                           0.82        90
   macro avg       0.81      0.75      0.77        90
weighted avg       0.82      0.82      0.81        90

